In [87]:
def categorical_crossentropy(t,y):
    return np.mean(-t*np.log(y+0.0001))

In [67]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [36]:
def softmax(x):
    c = np.max(x,axis=1).reshape(-1,1)
    x = x-c
    return np.exp(x)/np.sum(np.exp(x),axis=1).reshape(-1,1)

In [37]:
import numpy as np
class Relu:
    def __init__(self):
        self.mask = None
    def forward(self,x):
        self.mask = (x <=0)
        out = x.copy()
        out[self.mask] = 0
        return out
    def backward(self,dout):
        dout[self.mask] = 0
        dx = dout
        return dx
        
class Sigmoid:
    def __init__(self):
        self.out = None
    
    def forward(self,x):
        out = sigmoid(x)
        self.out = out
        return out
    
    def backward(self,dout):
        dx = ((1-self.out)*self.out)*dout
        return dx

class Affine:
    def __init__(self,W,b):
        self.W = W
        self.b = b
        self.x = None
        self.origin_shape = None
        self.dW = None
        self.db = None
    
    def forward(self,x):
        self.origin_shape = x.shape
        self.x = x
        out = np.dot(x,self.W) + self.b
        return out
    
    def backward(self,dout):
        dx = np.dot(dout,self.W.T)
        self.dW = np.dot(self.x.T,dout)
        self.db = np.sum(dout,axis=0)
        dx = dx.reshape(self.origin_shape)
        return dx

class Loss:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None
    
    def forward(self,t,y):
        self.y = softmax(y)
        self.t = t
        self.loss = categorical_crossentropy(self.t, self.y)
        return self.loss
    
    def backward(self,dout=1):
        dx = (self.y - self.t)*dout
        return dx

In [38]:
class Layers:
    def __init__(self):
        self.layers = {}
        
    def add(self,x1,x2,activation):
        activation_dict = {
            'sigmoid':Sigmoid,
            'relu':Relu,
            'softmax':Loss
        }
        w = np.random.randn(x1,x2)
        b = np.zeros(x2)
        activation_layer = 'activation'+str((int(len(self.layers)/2+1))) 
        Affine_layer = 'Affine'+str((int(len(self.layers)/2+1)))
        self.layers[Affine_layer] = Affine(w,b)
        self.layers[activation_layer] = activation_dict[activation]()
        
    
    def predict(self,x):
        out = x.copy()
        ind = 1 
        layer_len = len(self.layers)
        for key, layer in self.layers.items():
            if ind < layer_len :
                out = layer.forward(out)
            ind += 1
        return out
            
    
    def loss(self,x,t):
        y = self.predict(x)
        out = list(self.layers.values())[-1].forward(t,y)
        return out
    
    def accuracy(self,x,t):
        y = self.predict(x)
        y = np.argmax(y,axis=1)
        t = np.argmax(t,axis=1)
        self.acc = np.sum(y==t)/t.size
        return self.acc
    
    def gradient(self,x,t):
        self.loss(x,t)
        lr = 1e-4
        dout = 1
        dout = list(self.layers.values())[-1].backward(dout)
        layers = list(self.layers.values())[::-1][1:]
        self.layers_key = list(self.layers.keys())[::-1][1:]
        for layer in layers:
            dout = layer.backward(dout)
        self.grads = {}
        for layer_key in self.layers_key:
            if 'Affine' in layer_key:
                self.grads[layer_key] = [self.layers[layer_key].dW, self.layers[layer_key].db]
        for layer_key in self.layers_key:
            if 'Affine' in layer_key:
                self.layers[layer_key].W -= lr*self.grads[layer_key][0]
                self.layers[layer_key].b -= lr*self.grads[layer_key][1]
        result = self.loss(x,t)       
        return result
    
    def fit(self,x,t,epochs,lr):
        self.lr = lr
        self.history = {}
        accuracy = []
        loss = []
        for epoch in range(epochs):
            self.gradient(x,t)
            loss.append(self.err)
            accuracy.append(self.accuracy(x,t))
            if epoch % 100 == 0:
                print(f'loss : {self.err} === accuracy : {self.accuracy(x,t)}')
        self.history['accuracy'] = accuracy
        self.history['loss'] = loss
                

In [17]:
# 필요한 모듈 호출
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 텐서플로우 모듈
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils.np_utils import to_categorical
from tensorflow.keras.utils import plot_model
from keras.datasets import mnist

In [7]:
dir(keras.utils)

['CustomObjectScope',
 'GeneratorEnqueuer',
 'OrderedEnqueuer',
 'Progbar',
 'Sequence',
 'SequenceEnqueuer',
 'SidecarEvaluator',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_sys',
 'array_to_img',
 'custom_object_scope',
 'deserialize_keras_object',
 'experimental',
 'get_custom_objects',
 'get_file',
 'get_registered_name',
 'get_registered_object',
 'get_source_inputs',
 'image_dataset_from_directory',
 'img_to_array',
 'load_img',
 'model_to_dot',
 'normalize',
 'pack_x_y_sample_weight',
 'plot_model',
 'register_keras_serializable',
 'save_img',
 'serialize_keras_object',
 'set_random_seed',
 'text_dataset_from_directory',
 'timeseries_dataset_from_array',
 'to_categorical',
 'unpack_x_y_sample_weight']

In [74]:
# 데이터 정리
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [75]:
# 데이터 확인
print(x_train.shape)
print(x_test.shape)

(60000, 28, 28)
(10000, 28, 28)


In [90]:
# x 데이터 변환
x_train = x_train.reshape(-1,28 *28)
x_test = x_test.reshape(-1,28 *28)

In [77]:
np.unique(y_train).size

10

In [78]:
# y 데이터 변환 (one-hot인코딩)
y_train = to_categorical(y_train)

In [79]:
y_train.shape

(60000, 10)

In [81]:
model = Layers()

In [82]:
model.add(784, 1024, 'relu')
model.add(1024, 30, 'relu')
model.add(30, 10, 'softmax')

In [83]:
model.layers['Affine1'].W.shape

(784, 1024)

In [89]:
for i in range(10):
    print(model.gradient(x_train, y_train))

0.8312168928856521
0.8301116400416148
0.8313550494911567
0.3733792915345496
0.3317749117039264
0.30513996497269735
0.2827132213321223
0.2627541670752728
0.2468966947360632
0.23679112357539917


In [91]:
pred = model.predict(x_test)
pred = np.argmax(pred, axis=1)

In [92]:
np.sum(pred == y_test) / y_test.size

0.1135

In [93]:
y_test

array([7, 2, 1, ..., 4, 5, 6], dtype=uint8)